### prepare the env 

In [18]:
import os
from dotenv import load_dotenv, dotenv_values 
load_dotenv() 

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

### import libraries 

In [5]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import FewShotPromptTemplate , PromptTemplate

##### set embeddings 
###### I use this website to read about it 
###### [Google Generative AI Embeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_generative_ai/)

In [20]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

##### set Prompt Template

In [14]:
example_prompt = PromptTemplate(
    input_variables = ["input" , "output"],
    template = "Example Input : {input}\nExample Output : {output}"
    )

In [47]:
examples = [
    {"input" : "عصفور", "output" : "طائر"},
    {"input" : "قرش", "output" : "سمكة"},
    
]

##### set example selector 

###### [SemanticSimilarityExampleSelector Documentation](https://api.python.langchain.com/en/latest/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html)

In [51]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2
)

##### set similar prompt 

###### [FewShotPromptTemplate documentation](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html)

In [52]:
similar_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "بالاعتماد على الامثلة ,حاول اعطاء الاجابة المناسبة",
    suffix = "Input: {noun}\nOuput:",
    input_variables = ["noun"],
)

##### take a look on tamplet now 

In [53]:
print(similar_prompt.format(noun="اسد"))

بالاعتماد على الامثلة ,حاول اعطاء الاجابة المناسبة

Example Input : عصفور
Example Output : طائر

Example Input : قرش
Example Output : سمكة

Input: اسد
Ouput:


### test the model 

In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI  , HarmBlockThreshold, HarmCategory

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)


In [56]:
print(llm.invoke(similar_prompt.format(noun="اسد")).content)

Output: حيوان مفترس 



### Summary 

##### this way of creating template prompt make it ezy for developer to control the output of the model 